Alunos: Arthur Merode Germano (22103404), Pedro Mattia Lunelli (22106771), Vitor Luis Cavagnoli Machado(22106873)

<h1>Códigos referente a pergunta 1</h1>

In [ ]:
from pysus.online_data import SINAN, FTP_Inspect, parquets_to_dataframe
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
dfFinal = pd.DataFrame()
listAnos = [2019,2018,2017,2016,2015,2014,2013,2012]
for i in listAnos:
    df = parquets_to_dataframe(SINAN.download('Sífilis Adquirida',i))
    df.drop(columns=['TP_NOT','CS_SUSPEIT','SEM_NOT','ID_MUNICIP','DT_SIN_PRI','SEM_PRI','CS_GESTANT','SG_UF_NOT','ID_MN_RESI','ID_RG_RESI','ID_PAIS','NDUPLIC_N','IN_VINCULA','DT_INVEST','ID_OCUPA_N','CLASSI_FIN','CRITERIO','TPAUTOCTO','COUFINF','COPAISINF','COMUNINF',"DOENCA_TRA",'EVOLUCAO','DT_OBITO','DT_ENCERRA','TP_SISTEMA','TPUNINOT'],inplace=True)
    dfFinal = dfFinal.append(df)


In [ ]:
from pysus.preprocessing.decoders import decodifica_idade_SINAN
df = dfFinal
sem_idade = df.loc[df['NU_IDADE_N'] == '****']
df.drop(sem_idade.index, inplace=True)
df['IDADE'] = df['NU_IDADE_N'].apply(decodifica_idade_SINAN).fillna(-1).astype(int)
df = df.loc[(df['IDADE']>=25)&(df['IDADE']<=100)]

In [ ]:
# 1. 1ª a 4ª série incompleta do EF == EFI
# 2. 4ª série completa do EF ( antigo 1° grau) == EFI
# 3. 5ª à 8ª série incompleta do EF (antigo ginásio ou 1° grau) == EFI
# 4. Ensino fundamental completo (antigo ginásio ou 1° grau)== EFC
# 5. Ensino médio incompleto (antigo colegial ou 2° grau) == EMI
# 6. Ensino médio completo (antigo colegial ou 2° grau) == EMC
# 7. Educação superior incompleta == ESI
# 8. Educação superior completa == ESC
# 9. Ignorado == IGNORADO
# 10. Não se aplicavel idade < 7 anos == <7 ANOS
def anos_escolaridade(n):
    if  n in ['01','02','03']:
        return 'Ensino Fundamental Incompleto'
    elif n == '04':
        return'Ensino Fundamental Completo'
    elif n == '05':
        return 'Ensino Médio Incompleto'
    elif n == '06':
        return 'Ensino Médio Completo'
    elif n == '07':
        return 'Ensino Superior Incompleto'
    elif n == '08':
        return 'Ensino Superior Completo'
    else:
        return 'NAO INFORMADO'

df['NIVEL_ESCOLARIDADE'] = df['CS_ESCOL_N'].apply(anos_escolaridade)
df.to_csv('dados_analisados_1.csv',index=False)

In [ ]:
df2 = df.groupby('NIVEL_ESCOLARIDADE').size().reset_index(name='TOTAL')

def porcentagem(df):
    pessoas = [13449416,67667374.25,16811770,9456620.625,57580312.25,8405885,36565599.75]
    if df['NIVEL_ESCOLARIDADE'] == 'Ensino Fundamental Incompleto':
        return (df['TOTAL']/(pessoas[1]+pessoas[0]))*100
    elif df['NIVEL_ESCOLARIDADE'] == 'Ensino Fundamental Completo':
        return (df['TOTAL']/pessoas[2])*100
    elif df['NIVEL_ESCOLARIDADE'] == 'Ensino Médio Incompleto':
        return (df['TOTAL']/pessoas[3])*100
    elif df['NIVEL_ESCOLARIDADE'] == 'Ensino Médio Completo':
        return (df['TOTAL']/pessoas[4])*100
    elif df['NIVEL_ESCOLARIDADE'] == 'Ensino Superior Incompleto':
        return (df['TOTAL']/pessoas[4])*100
    elif df['NIVEL_ESCOLARIDADE'] == 'Ensino Superior Completo':
        return (df['TOTAL']/pessoas[6])*100
    else:
        return (df['TOTAL']/pessoas[0])*100

df2 = df2.loc[df2['NIVEL_ESCOLARIDADE']!='NAO INFORMADO']
df2['PORCENTAGEM'] = df2.apply(porcentagem,axis=1)
ordem = [1, 0, 3, 2, 5, 4]
df2 = df2.iloc[ordem]
df2.reset_index(drop=True, inplace=True)

In [ ]:
data = {
    'NIVEL_ESCOLARIDADE': ['Ensino Fundamental Incompleto', 'Ensino Fundamental Completo', 'Ensino Médio Incompleto', 'Ensino Médio Completo', 'Ensino Superior Incompleto', 'Ensino Superior Completo'],
    'PORCENTAGEM': [38.6, 8, 4.5, 27.4, 4, 17.4]
}

dfAux = pd.DataFrame(data, columns=['NIVEL_ESCOLARIDADE', 'PORCENTAGEM'])

In [ ]:
import numpy as np
x =np.arange(6)
largura = 0.40
eixo_xA = x - largura/2
eixo_xB = x + largura/2+0.01
#plot do gráfico de barras
color = ['#e5c494','#ffd92f','#a6d854','#e78ac3','#8da0cb','#fc8d62']
soma = df2["PORCENTAGEM"].sum()
for index, row in df2.iterrows():
    percentage = (row["PORCENTAGEM"] /soma) * 100
    df2.at[index, 'PORCENTAGEM'] = percentage
fig,ax = plt.subplots(figsize = (12,6))
principal = ax.bar(eixo_xA,df2['PORCENTAGEM'],largura,color = color,edgecolor='black')
sec = ax.bar(eixo_xB,dfAux['PORCENTAGEM'],largura)
for index, row in df2.iterrows():
    percentage = f'{row["PORCENTAGEM"] :.2f}%'
    ax.text(index, row["PORCENTAGEM"], percentage, ha='right', va='bottom')
for index, row in dfAux.iterrows():
    ax.text(index+0.05, row["PORCENTAGEM"], f'{row["PORCENTAGEM"]:.2f}%', ha='left', va='bottom')
ax.set_xticks(x)
ax.set_xticklabels(df2['NIVEL_ESCOLARIDADE'],rotation=40, ha="right")
plt.legend((principal[0], sec[0]), ('Casos por nível de instrução', 'Proporção do nível de instrução da pop. brasileira'),loc = 'upper right',title='Legenda')
plt.title('Proporção de pessoas com Sífilis Adquirida por Nível de Escolaridade')
plt.xlabel('Nível de Escolaridade')
plt.ylabel('Porcentagem')
plt.show()  



<h1>Códigos referentes a pergunta 2</h1>

In [ ]:
from pysus.online_data import SINAN, FTP_Inspect, parquets_to_dataframe
import pandas as pd
from pysus.preprocessing.decoders import decodifica_idade_SINAN
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
SINAN.get_available_years('Sífilis Adquirida') #disponibilidade de dados por ano

In [ ]:
dfFinal = pd.DataFrame()
listAnos = [2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010] #anos que serão escolhidos para a análise

for i in listAnos: #Loop para baixar os arquivos de cada ano de acordo com a lista
    df = parquets_to_dataframe(SINAN.download('Sífilis Adquirida',i))#DOWNLOAD
    df.drop(columns=['TP_NOT','CS_SUSPEIT','SEM_NOT','ID_MUNICIP','DT_SIN_PRI','SEM_PRI','CS_GESTANT','SG_UF_NOT','ID_MN_RESI','ID_RG_RESI','ID_PAIS','NDUPLIC_N','IN_VINCULA','DT_INVEST','ID_OCUPA_N','CLASSI_FIN','CRITERIO','TPAUTOCTO','COUFINF','COPAISINF','COMUNINF',"DOENCA_TRA",'EVOLUCAO','DT_OBITO','DT_ENCERRA','TP_SISTEMA','TPUNINOT'],inplace=True) #drop de colunas "inúteis"
    dfFinal = dfFinal.append(df) #junção dos dataframes de cada ano em um único dataframe
dfFinal.to_csv('dados_analisados_2.csv') #salvando o dataframe em um arquivo csv para enviar ao professor, sobre os dados analisados

byYear = dfFinal.groupby(['NU_ANO']).size() # agrupamento por ano para identificar a quantidade de casos por ano

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure(figsize=(10, 6))  # Definir tamanho da figura (largura, altura)

ax = sns.barplot(x=byYear.index, y=byYear.values, color='red', hue=byYear.values, edgecolor='black', linewidth=1.5, saturation=1, dodge=False) # plotagem do gráfico de barras

plt.xlabel('Ano') # nome do eixo x
plt.ylabel('Quantidade de casos') # nome do eixo y
plt.title('Quantidade de casos de Sífilis Adquirida por ano') # título do gráfico

# Adicionando os valores das barras
for i, v in enumerate(byYear.values):
    ax.text(i, v, '{:,.0f}'.format(v).replace(',', '.'), color='black', ha='center', va='bottom', fontsize=10)  # Aumentar o tamanho da fonte para fontsize=10

# Removendo a barra preta da legenda
handles, labels = ax.get_legend_handles_labels()
handles = handles[:len(byYear.index)]
labels = list(byYear.index)
ax.legend(handles=handles, labels=labels, loc='upper left')

# Ajustando o separador de milhar no eixo y
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, p: format(int(x), ',').replace(',', '.')))

# Aumentar a distância entre as barras
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

plt.tight_layout()  # Ajustar layout da figura

plt.show() # plotagem do gráfico
